In [ ]:
from flask import Flask, jsonify, request
from werkzeug.utils import secure_filename
from flask import render_template
from flask import make_response
from FruitsNeuralNetwork import FruitsNeuralNetwork
import torch
from time import time
from PIL import Image
import json
import numpy as np

# Consultar nombre del modelo
json_params= open('params.json')
params_dict = json.load(json_params)
model_name = params_dict["model_name"]

# Cargamos el modelo
model = FruitsNeuralNetwork()
model.load_state_dict(torch.load(model_name))
model.eval()

# definimos la app
app = Flask(__name__, template_folder='web',
            static_url_path='/static',
            static_folder='web/static',)

@app.route("/")
def hello():
    version=int(time());
    return render_template("index.html",
                            version=version)

@app.route('/analize', methods = ['POST', 'GET'])
def upload_file():
    # Validar que se esté subiendo algo
    if request.method == 'POST':
          # Recorremos los archivos
          for i in request.files:
                # Abrimos la imagen y la formateamos al tamaño adecuado
                img = Image.open(request.files[i])
                img.resize(100, 100)
                img_tensor = np.asarray(img)
                outputs = model(img_tensor)
                return outputs
if __name__ == "__main__":
  app.run()